In [7]:
%run 0.0_Dependance.ipynb

In [8]:
df = pd.read_csv("train.csv",index_col=0)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)

#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])

target = df.Status
data = df.drop(columns=["Status","N_Days"])

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=123)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='not majority',random_state=42)

X_train, y_train = smote.fit_resample(X_train, y_train)

In [12]:
y_train.value_counts()

Status
0    3999
2    3999
1    3999
Name: count, dtype: int64

In [10]:
from joblib import load, dump

df_score_best_test = load("df_score_best_test.joblib")
model = XGBClassifier()
comparaison = []
for j,i in enumerate(df_score_best_test["Best_param"]):
    best_model=XGBClassifier(**i)
    best_model.fit(X_train,y_train)
    probs = best_model.predict_proba(X_test)
    logloss = log_loss(y_test, probs)
    delta = logloss-df_score_best_test["LogLoss"].values[j]
    comparaison.append(delta)

In [11]:
comparaison

[0.023851338180844672,
 0.0125821282409872,
 0.03404464818957437,
 0.03794883075012917,
 0.02209329424307338]